In [2]:
# Load data and set pandas options to display the full dataset if needed
import re
import pandas as pd
from Notebooks import utils

pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_colwidth', -1)  
restaurant_data = pd.read_csv('../Data/restaurants.tsv', sep='\t')
restaurant_data = restaurant_data.set_index(['id'], drop=False)
restaurant_data.head()
# Load the gold standard duplicates to calculate metrics
true_duplicates = pd.read_csv('../Data/restaurants_DPL.tsv', sep='\t')

In [3]:
print('Duplicates without any preprocessing')
duplicates_bool = utils.get_duplicates_bool(restaurant_data)

Duplicates without any preprocessing
Duplicates in address: 92
Duplicates in name: 88
Duplicates in phone: 7
Duplicates in city: 815
Duplicates in address name and phone: 0
Duplicates in address city and name: 24
Duplicates in name city and phone0
Duplicates in address, city, name and phone: 0
duplicates in 3 columns of address, city, name and phone24


In [4]:
print('Metrics without any preprocessing')
detected_duplicates = restaurant_data[duplicates_bool == True]
utils.print_metrics(restaurant_data, detected_duplicates, true_duplicates)

Metrics without any preprocessing
All entries in original dataset: 864
Detected duplicates (all): 24
Real duplicates (from gold standard): 112
True positives: 24
True negatives: 752
False positives: 0
False negatives: 88
Accuracy 0.8981481481481481
Precision: 1.0
Recall: 0.21428571428571427


In [5]:
# Get info about the data and datatypes of the DataFrame
restaurant_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 864 entries, 1 to 864
Data columns (total 6 columns):
id         864 non-null int64
name       864 non-null object
address    864 non-null object
city       864 non-null object
phone      864 non-null object
type       863 non-null object
dtypes: int64(1), object(5)
memory usage: 47.2+ KB


In [6]:
# Remove special characters from phone, address, name and city to detect duplicates
restaurant_data = utils.remove_special_characters(restaurant_data)
restaurant_data = utils.trim_multiple_blanks(restaurant_data)

In [7]:
# Remove the direction in address and name because it's inconsistent over the dataset
direction_regex = re.compile('(( |^)((south)|(east)|(west)|(north)|(ne)|(se)|(nw)|(sw)|s|w|e|n)( |$))')
restaurant_data.address = restaurant_data.address.map(lambda x: direction_regex.sub(' ', x))
restaurant_data.name = restaurant_data.name.map(lambda x: direction_regex.sub(' ', x))
restaurant_data.city = restaurant_data.city.map(lambda x: direction_regex.sub(' ', x))
restaurant_data = utils.trim_multiple_blanks(restaurant_data)

In [8]:
print('Duplicates after removing generic clearing')
duplicates_bool = utils.get_duplicates_bool(restaurant_data)

Duplicates after removing generic clearing
Duplicates in address: 103
Duplicates in name: 91
Duplicates in phone: 116
Duplicates in city: 815
Duplicates in address name and phone: 51
Duplicates in address city and name: 26
Duplicates in name city and phone41
Duplicates in address, city, name and phone: 25
duplicates in 3 columns of address, city, name and phone68


In [9]:
print('Metrics after generic clearing')
detected_duplicates = restaurant_data[duplicates_bool == True]
utils.print_metrics(restaurant_data, detected_duplicates, true_duplicates)

Metrics after generic clearing
All entries in original dataset: 864
Detected duplicates (all): 68
Real duplicates (from gold standard): 112
True positives: 68
True negatives: 752
False positives: 0
False negatives: 44
Accuracy 0.9490740740740741
Precision: 1.0
Recall: 0.6071428571428571


In [10]:
# Map multiple occurrences of the same city in different writing
# It could also be a good idea to map all city parts to one city (i.e. hollywood -> los angeles)
city_map = {'la': 'los angeles', 'new york city': 'new york'}
restaurant_data.city = restaurant_data.city.replace(city_map)

In [11]:
print('Duplicates after clearing city')
duplicates_bool = utils.get_duplicates_bool(restaurant_data)

Duplicates after clearing city
Duplicates in address: 103
Duplicates in name: 91
Duplicates in phone: 116
Duplicates in city: 817
Duplicates in address name and phone: 51
Duplicates in address city and name: 49
Duplicates in name city and phone78
Duplicates in address, city, name and phone: 48
duplicates in 3 columns of address, city, name and phone82


In [12]:
print('Metrics after clearing city')
detected_duplicates = restaurant_data[duplicates_bool == True]
utils.print_metrics(restaurant_data, detected_duplicates, true_duplicates)

Metrics after clearing city
All entries in original dataset: 864
Detected duplicates (all): 82
Real duplicates (from gold standard): 112
True positives: 82
True negatives: 752
False positives: 0
False negatives: 30
Accuracy 0.9652777777777778
Precision: 1.0
Recall: 0.7321428571428571


In [13]:
print('length of unique addresses before clearing ' + str(len(restaurant_data.address.unique())))

length of unique addresses before clearing 761


In [14]:
# Remove unnecessary explaination parts from the address string for a more accurate duplicate detection
restaurant_data['address'] = restaurant_data['address'].str.split(r' between| off| near| at| in').str[0]

In [15]:
# Remove appendixes of numbers because they are inconsistent over the dataset
restaurant_data.address = restaurant_data.address.map(lambda x: re.sub(r"(?<=\d)(st|nd|rd|th)\b", '', x))

In [16]:
# Standardize the address even more
address_num_map = {'first': '1', 'second': '2', 'third': '3', 'fourth': '4', 'fifth': '5', 'sixth': '6',
                   'seventh': '7','eighth' : '8', 'ninth': '9', 'tenth': '10', 'eleventh': '11', 'twelfth': '12'}
address_name_map = {'la': 'los angeles', 'ave': 'avenue',  'rd': 'road', 'blv': 'boulevard', 
                    'blvd': 'boulevard', 'st': 'street'}
address_map = {**address_name_map, **address_num_map}
restaurant_data.address = restaurant_data.address.map(lambda x: ' '.join([address_map.get(i, i) for i in x.split()]))

In [17]:
print('Unique addresses after clearing ' + str(len(restaurant_data.address.unique())))

Unique addresses after clearing 739


In [18]:
print('Duplicates after clearing the address')
duplicates_bool = utils.get_duplicates_bool(restaurant_data)

Duplicates after clearing the address
Duplicates in address: 125
Duplicates in name: 91
Duplicates in phone: 116
Duplicates in city: 817
Duplicates in address name and phone: 68
Duplicates in address city and name: 67
Duplicates in name city and phone78
Duplicates in address, city, name and phone: 65
duplicates in 3 columns of address, city, name and phone83


In [19]:
print('Metrics after clearing the address')
detected_duplicates = restaurant_data[duplicates_bool == True]
utils.print_metrics(restaurant_data, detected_duplicates, true_duplicates)

Metrics after clearing the address
All entries in original dataset: 864
Detected duplicates (all): 83
Real duplicates (from gold standard): 112
True positives: 83
True negatives: 752
False positives: 0
False negatives: 29
Accuracy 0.9664351851851852
Precision: 1.0
Recall: 0.7410714285714286


In [20]:
print('Unique names before clearing name: ' + str(len(restaurant_data.name.unique())))

Unique names before clearing name: 773


In [21]:
unique_city_list = restaurant_data.city.unique()
restaurant_data.name = restaurant_data.name.str.split(r' between| off| near| at| in| of').str[0]
city_regex = re.compile('the|restaurant|and|new york city|' +'|'.join(map(re.escape, unique_city_list)))
restaurant_data.name = [city_regex.sub('', name) for name in restaurant_data.name]

In [22]:
restaurant_data.name = restaurant_data.name.apply(lambda x: ' '.join(sorted(x.split(' '))))
restaurant_data = utils.trim_multiple_blanks(restaurant_data)

In [23]:
print('Unique addresses after clearing name: ' + str(len(restaurant_data.address.unique())))

Unique addresses after clearing name: 739


In [24]:
print('Duplicates after clearing name')
duplicates_bool = utils.get_duplicates_bool(restaurant_data)

Duplicates after clearing name
Duplicates in address: 125
Duplicates in name: 105
Duplicates in phone: 116
Duplicates in city: 817
Duplicates in address name and phone: 76
Duplicates in address city and name: 73
Duplicates in name city and phone84
Duplicates in address, city, name and phone: 71
duplicates in 3 columns of address, city, name and phone91


In [25]:
print('Metrics after clearing')
detected_duplicates = restaurant_data[duplicates_bool == True]
detected_non_duplicates = restaurant_data[duplicates_bool == False]
utils.print_metrics(restaurant_data, detected_duplicates, true_duplicates)

Metrics after clearing
All entries in original dataset: 864
Detected duplicates (all): 91
Real duplicates (from gold standard): 112
True positives: 91
True negatives: 752
False positives: 0
False negatives: 21
Accuracy 0.9756944444444444
Precision: 1.0
Recall: 0.8125


In [26]:
# Check the dataset without duplicates
restaurant_data[restaurant_data['id'].isin(detected_duplicates['id'])== False].head(100)

,id,name,address,city,phone,type
id,,,,,,
1,1,arnie morton,435 los angeles cienega boulevard,los angeles,3102461501,american
3,3,art delicatessen,12224 ventura boulevard,studio city,8187621221,american
4,4,art deli,12224 ventura boulevard,studio city,8187621221,delis
5,5,belair hotel,701 stone canyon road,bel air,3104721211,californian
7,7,bizou cafe,14016 ventura boulevard,sherman oaks,8187883536,french
9,9,campanile,624 los angeles brea avenue,los angeles,2139381447,american
11,11,chinois main on,2709 main street,santa monica,3103929025,french
13,13,citrus,6703 melrose avenue,los angeles,2138570034,californian
15,15,fenix,8358 sunset boulevard,hollywood,2138486677,american


In [27]:
 
utils.get_final_dataset(_, restaurant_data)

dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found
dup found


{'1': [2],
 '10': [],
 '100': [],
 '101': [102],
 '102': [],
 '105': [106],
 '106': [],
 '107': [108],
 '108': [],
 '109': [110],
 '11': [12],
 '110': [],
 '111': [112],
 '112': [],
 '113': [114],
 '114': [],
 '115': [116],
 '116': [],
 '117': [118],
 '118': [],
 '119': [120],
 '12': [],
 '120': [],
 '123': [124],
 '124': [],
 '125': [126],
 '126': [],
 '127': [128],
 '128': [],
 '129': [130],
 '13': [14],
 '130': [],
 '131': [132],
 '132': [],
 '135': [136],
 '136': [],
 '14': [],
 '141': [142],
 '142': [],
 '149': [150],
 '15': [16],
 '150': [],
 '151': [152],
 '152': [],
 '155': [156],
 '156': [],
 '157': [158],
 '158': [],
 '16': [],
 '161': [162],
 '162': [],
 '163': [164],
 '164': [],
 '165': [166],
 '166': [],
 '167': [168],
 '168': [],
 '169': [170],
 '17': [18],
 '170': [],
 '171': [172],
 '172': [],
 '173': [174],
 '174': [],
 '175': [176],
 '176': [],
 '179': [180],
 '18': [],
 '180': [],
 '181': [182],
 '182': [],
 '183': [184],
 '184': [],
 '185': [186],
 '186': [],
 '187'

In [28]:
#from pymongo import MongoClient
#client = MongoClient("mongodb+srv://Username:PW@cluster0-uqgxm.mongodb.net/test?retryWrites=true&w=majority", serverSelectionTimeoutMS=60)
#db = client.dmdb_db
#collection = db.testet

#data = restaurant_data.to_dict(orient='records') 
#data = {"keay": "vaasfewio"}
#collection.insert_one(data)